# This notepad demonstrates the usage Open AI's LLM model and Langgraph Agent Framework for performing sentiment analysis on financial news articles.

### Install required packages

In [134]:
!pip install -q langgraph langchain_openai

### Import required modules

In [135]:
import os
from google.colab import userdata
from langgraph.graph import StateGraph, START,END
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from typing import Dict, List, TypedDict, Any

### Get the api_key and initilize the model

In [136]:
os.environ['OPENAI_API_KEY']=userdata.get('OPENAI_API_KEY')

In [137]:
model=ChatOpenAI(temperature=0, model="gpt-4o-mini")

### Define a class with the required variables for the graph

In [178]:
class newsState(TypedDict):
  user_input: str
  news_sentiment: str
  user_output: str
  messages: List[Dict[str,Any]]

### Define a function for performing sentiment analysis on the input news article for the graphs node.

In [179]:
def perform_sentiment_analysis(newsState):
  """Analyze financial news and perform sentiment analysis."""

  prompt=f"""You are a financial news sentiment analysis expert. Analyze the sentiment of the following financial news article and classify it as either positive, negative, or neutral.

  {newsState['user_input']}

  Respond with the sentiment only. Do not explain your reasoning."""

  messages=[HumanMessage(content=prompt)]

  response = model.invoke(messages)

  new_messages=[{'role':'user', 'content' : prompt}, {'role':'assistant', 'content':response.content}]

  return {'news_sentiment' : response.content, 'messages' : new_messages}

### Define a function for providing the sentiment as an output.

### Define the graph with the required nodes and edges

In [181]:
newsGraph=StateGraph(newsState)
newsGraph.add_node("perform_sentiment_analysis",perform_sentiment_analysis)
newsGraph.add_node("draft_user_response",draft_user_response)
newsGraph.add_edge(START,"perform_sentiment_analysis")
newsGraph.add_edge("perform_sentiment_analysis","draft_user_response")
newsGraph.add_edge("draft_user_response",END)
compiled_graph=newsGraph.compile()

### Verify the output

In [183]:
print(output_news_sentiment)

{'user_input': 'Stock Radar: LTIMindtree gives a breakout from falling wedge pattern; time to buy after over 30% fall from highs.\nLTIMindtree Ltd, after a 32% fall, shows signs of a bullish comeback, breaking out from a falling wedge pattern.\nExperts suggest short-term traders can buy with a target of Rs 4,820 in 3-4 weeks, supported by positive technical indicators like Supertrend buy and RSI divergence.\nA stop loss at Rs 4,300 is recommended.', 'news_sentiment': 'Positive', 'user_output': 'Thanks for asking! \n\nHere are the details: \n\nNews Article: Stock Radar: LTIMindtree gives a breakout from falling wedge pattern; time to buy after over 30% fall from highs.\nLTIMindtree Ltd, after a 32% fall, shows signs of a bullish comeback, breaking out from a falling wedge pattern.\nExperts suggest short-term traders can buy with a target of Rs 4,820 in 3-4 weeks, supported by positive technical indicators like Supertrend buy and RSI divergence.\nA stop loss at Rs 4,300 is recommended. \

In [185]:
input_news="""Stock Radar: LTIMindtree gives a breakout from falling wedge pattern; time to buy after over 30% fall from highs.
LTIMindtree Ltd, after a 32% fall, shows signs of a bullish comeback, breaking out from a falling wedge pattern.
Experts suggest short-term traders can buy with a target of Rs 4,820 in 3-4 weeks, supported by positive technical indicators like Supertrend buy and RSI divergence.
A stop loss at Rs 4,300 is recommended."""

output_news_sentiment=compiled_graph.invoke({'user_input':input_news})
print(output_news_sentiment['user_output'])

Thanks for asking! 

Here are the details: 

News Article: Stock Radar: LTIMindtree gives a breakout from falling wedge pattern; time to buy after over 30% fall from highs.
LTIMindtree Ltd, after a 32% fall, shows signs of a bullish comeback, breaking out from a falling wedge pattern.
Experts suggest short-term traders can buy with a target of Rs 4,820 in 3-4 weeks, supported by positive technical indicators like Supertrend buy and RSI divergence.
A stop loss at Rs 4,300 is recommended. 

Sentiment: Positive 

Hope you are happy with the response.


In [187]:
input_news="""Gautam Adani’s kin Pranav Adani, accused of insider trading by SEBI, says ‘seeking to settle’
Pranav Adani reportedly shared unpublished price-sensitive information regarding Adani Green’s 2021 acquisition of SB Energy Holdings with his brother-in-law Kunal Shah before the deal was made public."""

output_news_sentiment=compiled_graph.invoke({'user_input':input_news})
print(output_news_sentiment['user_output'])

Thanks for asking! 

Here are the details: 

News Article: Gautam Adani’s kin Pranav Adani, accused of insider trading by SEBI, says ‘seeking to settle’
Pranav Adani reportedly shared unpublished price-sensitive information regarding Adani Green’s 2021 acquisition of SB Energy Holdings with his brother-in-law Kunal Shah before the deal was made public. 

Sentiment: Negative 

Hope you are happy with the response.
